In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from datetime import timedelta
from notebook_utils import NotebookUtilities
from pandas import DataFrame
import humanize
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re

nu = NotebookUtilities(data_folder_path=os.path.abspath('../data'))

In [3]:

# Get all CSVs into one data frame
if nu.pickle_exists('frvrs_logs_df'):
    frvrs_logs_df = nu.load_object('frvrs_logs_df')
    print(frvrs_logs_df.shape)
    # df = frvrs_logs_df.sample(4).dropna(axis='columns', how='all')
    # display(df.T)

(832366, 109)



# Gaze and Intent

Is there a correlation between gaze and intent, i.e., if a learner sees bleeding, do they treat it?

In [4]:

mask_series = (frvrs_logs_df.action_type == 'PLAYER_GAZE')
df = frvrs_logs_df[mask_series]
df.sample(min(4, df.shape[0])).dropna(axis='columns', how='all').T

,839495,841901,841815,834043
action_type,PLAYER_GAZE,PLAYER_GAZE,PLAYER_GAZE,PLAYER_GAZE
elapsed_time,281379,91082,72791,82630
event_time,2023-09-11 13:49:46,2023-09-07 14:50:24,2023-09-07 14:50:05,2023-09-11 10:11:45
session_uuid,d9d58da9-9bdc-41ea-90fe-4c84db4635d9,fde2c201-36fc-4123-bf42-f1bff1b24969,fde2c201-36fc-4123-bf42-f1bff1b24969,b11f14c2-8de9-4247-9753-1434cb392804
file_name,v.1.3/d9d58da9-9bdc-41ea-90fe-4c84db4635d9.csv,v.1.3/fde2c201-36fc-4123-bf42-f1bff1b24969.csv,v.1.3/fde2c201-36fc-4123-bf42-f1bff1b24969.csv,v.1.3/b11f14c2-8de9-4247-9753-1434cb392804.csv
logger_version,1.3,1.3,1.3,1.3
time_group,1,1,1,1
player_gaze_location,"(0.0, 0.0, 6.0)","(-4.2, 0.2, -21.5)","(-4.2, 0.2, -21.5)","(2.9, 0.0, 2.5)"
player_gaze_patient_id,Lily_0 Root,Military Mike Jungle Combat_3_2 Root,Military Mike Jungle Combat_3_2 Root,Tutorial Military Marine Root
player_gaze_distance_to_patient,1.20572,1.499485,2.240029,2.111216


In [5]:

# Iterate over the patient ID and get all the bleeding
if nu.pickle_exists('bleeding_treated_df'):
    bleeding_treated_df = nu.load_object('bleeding_treated_df')
else:
    groupby_columns = ['session_uuid', 'time_group', 'patient_id']
    gb = frvrs_logs_df.sort_values(['elapsed_time']).groupby(groupby_columns)
    rows_list = []
    for (session_uuid, time_group, patient_id), df1 in gb:
        
        # Add the logger version and the groupby columns to the row dictionary
        row_dict = {}
        logger_version = df1.logger_version.unique().item()
        row_dict['logger_version'] = logger_version
        for cn in groupby_columns: row_dict[cn] = eval(cn)
        
        # Did the trainee gaze at this patient at least once?
        mask_series = (df1.action_type == 'PLAYER_GAZE')
        row_dict['gazed_at_patient'] = bool(df1[mask_series].shape[0])
        
        # Is this patient bleeding?
        mask_series = df1.injury_treated_required_procedure.isin(['tourniquet', 'woundpack'])
        row_dict['is_patient_bleeding'] = bool(df1[mask_series].shape[0])
        
        # Was the bleeding treated?
        mask_series &= (df1.injury_treated_injury_treated == True)
        row_dict['bleeding_treated'] = bool(df1[mask_series].shape[0])
        
        # Add the row dictionary to the list
        rows_list.append(row_dict)
    
    # Create a data frame from the list of row dictionaries
    bleeding_treated_df = DataFrame(rows_list)
    nu.store_objects(bleeding_treated_df=bleeding_treated_df)

Pickling to C:\Users\DaveBabbitt\Documents\GitHub\itm-analysis-reporting\saves\pkl\bleeding_treated_df.pkl


In [6]:

# Display the minority combinations
groupby_columns = ['gazed_at_patient', 'is_patient_bleeding', 'bleeding_treated']
df = nu.get_minority_combinations(bleeding_treated_df, groupby_columns)
display(df.T)

,9755,8279,3775,3857
logger_version,1.3,1.3,1.3,1.3
session_uuid,fde2c201-36fc-4123-bf42-f1bff1b24969,d9d58da9-9bdc-41ea-90fe-4c84db4635d9,6037b1d3-eff2-4317-9372-768236fbae91,61dba1bb-2326-48f9-968f-9e0e4c867eee
time_group,1,1,1,2
patient_id,Intelligence Officer Burned_Gary_1 Root,Lily_11 Root,Helga_0 Root,Lily_2 Root
gazed_at_patient,True,True,False,False
is_patient_bleeding,False,True,True,True
bleeding_treated,False,True,False,True


In [15]:

groupby_columns = ['gazed_at_patient', 'is_patient_bleeding', 'bleeding_treated']
srs = bleeding_treated_df.groupby(groupby_columns).size()

# Get the count of cases where all conditions are met
count_true_true_true = srs.get((True, True, True), 0)

# Get the count of cases where gazed_at_patient is True and is_patient_bleeding is True
count_true_true = srs.get((True, True, False), 0) + srs.get((True, True, True), 0)

# Calculate the percentage
percentage = (count_true_true_true / count_true_true) * 100 if count_true_true > 0 else 0
if percentage == 100:
    yesno = 'Yes'
    percentage = f'all {count_true_true_true}'
elif percentage < 100:
    yesno = 'Not quite'
    percentage = f'{percentage:.1f}%'
elif percentage == 0:
    yesno = 'No'
    percentage = 'none'

print(f'{yesno}, {percentage} of the trainees who gazed at a patient and discovered they were bleeding, treated the patient.')

Yes, all 20 of the trainees who gazed at a patient and discovered they were bleeding, treated the patient.
